# AlgotradingPy Jupyter Notebook
## Autor: Daniel Antonio Tell
<br>


<div align="center">
Formato do arquivo CSV: <br>
      
|time            |M x M |bool |M x Preço|bool|RSI QS/QI    |bool |RSI OUT-S/OUT-I|bool |variação5|variação10|variação50|
|----------------|------|-----|---------|----|-------------|-----|---------------|-----|--------|--------|--------|
|2019-05-12 10:00|MC>=ML|false|Preço>MC |false|sobrecomprado|false|  sobrevendido |false | -0.51%  | -2%  | -1%  |
|2019-05-12 10:15|MC<ML |true |Preço>MC |false|sobrevendido |true|  sobrevendido |false | 0.22%    | -1%  | -10%  |

<br>
<div align="center">
Componentes de customização <i>(Widgets)<i>:<br>

|1º coluna|2º coluna|3º coluna|4º coluna|
|----|---|-----|----|
|data inicial {dtp}|data final {dtp}|timeframe {drp_dwn}|moedas {drp_dwn}|

|1º coluna|2º coluna|3º coluna|4º coluna|5º coluna|6º coluna|
|----|---|-----|----|----|----|
|Cruzamento de Médias {lbl}|-|-|Cruzamento de Média e Preço{lbl}|-|-|
|Media Curta {txt}|Media Longa {txt} | Stop Móvel {chk_box} |Media Curta {txt}|Stop Móvel {chk_box} |-|
|Stop gain {flt_txt}|stop loss {flt_txt} | Ganhos(%) {txt}|Stop gain {flt_txt}|stop loss {flt_txt}|Ganhos(%) {txt}|
|RSI {lbl}|-|-|Setups, Precisão e Exportação{lbl}|-|-|
|RSI Periodos {txt}| RSI Min {txt} | RSI Max {txt}|Compra (%) {f_txt}|Venda (%) {flt_txt}| Wallet (USD) {txt}|
|Ganhos(%) {txt}|-|-|Buy Hold {txt}|-|-|
|Ganhos C1 {txt}|Ganhos C2 {txt}|Ganhos C3 {txt}|Precisão (%) {lbl}|-|-|
|Legenda|-|-|MAxMA {txt}|MAxPreço{txt}|RSI (fixo) {txt}
|-|-|-|C1 {txt}|C2 {txt}|C3 {txt}|

|1º coluna|2º coluna|
|----|---|
|Executar Backtest {btn}|
</div> 

In [1]:
from algotradingpy.controller.backtest import BackTest
from algotradingpy.model.btresult import BacktestResult
from algotradingpy.controller.data import Mysql
from algotradingpy.model.plot import TimeSeries
from algotradingpy.model.setup import Setup
import algotradingpy.utils.config as config
import algotradingpy.view.console as console
import algotradingpy.utils.util as util

from datetime import date, datetime, timedelta
import time
import sys, os
import traceback
import logging
from pandas import DataFrame, concat
import pandas as pd
import matplotlib.dates as mdates
#import seaborn as sns
from ipywidgets import widgets, HBox, VBox, Layout, GridBox, ButtonStyle
from IPython.display import display, clear_output, FileLink, FileLinks
import json
from json import JSONDecodeError
%matplotlib inline



/home/daniel/CodigosFontes/Python/AlgoTradingPy/venv/lib/python3.10/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [2]:
# Programa Principal  
description = ""
df = None
df_trade = None
time_frames = ['1m', '5m', '15m', '30m', '1h', '3h', '6h', '12h', '1D']
coins = ['btcusd','ethusd', 'solusd', 'xrpusd', 'ltcusd', 'bchn:usd', 'trxusd', 'eosusd', 'neousd', 'xmrusd']


In [3]:
def do_log(msg, exception):
    print('\033[7;31m{}\033[m\n{}'.format(msg, exception))
    #logger.error(msg + '\n' + str(excep
    
# Funções de evento para os botões
def enable_buttons(enable):
    update_btn.disabled = not enable

# Análise e representação dos dados em forma de gráfico
def execute_backtest(asset, setup, start_date, end_date, back_test_days):
    #Chama a função para fazer o cruzamento das médias móveis curtas e longas
    backtest = BackTest(asset, setup, back_test_days)
    console.show("-" * 60)
    console.show(
        f"Retorno trade {backtest.bt_result.get_trade_returns():.2f}%, "
        f"buy_hold {backtest.bt_result.get_buy_and_hold_returns():.2f}%; "
        f"Precisão: {backtest.bt_result.get_accuracy():.2f}% ")
    console.show(
        f"Último sinal: {backtest.get_json_signal(False, 1)}")
    df_signals = pd.DataFrame(backtest.get_json_signal(True, 0))
    df_trades_buy = df_signals[df_signals['obs'].str.contains('Comprou')]
    df_trades_sell = df_signals[df_signals['obs'].str.contains('Vendeu')]
    console.show(
         f"Trades: compras {len(df_trades_buy)} vendas {len(df_trades_sell)} total {len(df_signals)}")
    console.show("-" * 60)
    console.show("")
    strategy = setup.get_strategy()
    
    # atualiza os componentes(widgets) com os ganhos calculados
    buy_hold_gain_txt.value = '{:.2f}'.format(backtest.bt_result.get_buy_and_hold_returns())
    if (setup.get_strategy() == 'MAxMA'):
        strategy = f'{setup.get_strategy()}_ema={setup.short}_sma={setup.long}'
        trade_gain_mm_txt.value = '{:.2f}'.format(backtest.bt_result.get_trade_returns())
        accuracy_ma_txt.value = '{:.2f}'.format(backtest.bt_result.get_accuracy())         
        # Chama a função para plotar as séries temporais do cruzamento das médias móveis curtas e longas
        console.show(backtest.get_short_ema()[1:2].index.time)
        timeSeries = TimeSeries(backtest)
        timeSeries.time_series_all(start_date, end_date, backtest.get_short_ema(), backtest.get_long_sma(), str(setup.short), str(setup.long))  
    elif (setup.get_strategy() == 'MAxPrice'):
        strategy = f'{setup.get_strategy()}_ema={setup.short}'
        trade_gain_mp_txt.value = '{:.2f}'.format(backtest.bt_result.get_trade_returns()) 
        accuracy_price_txt.value = '{:.2f}'.format(backtest.bt_result.get_accuracy())         
        # Chama a função para plotar as séries temporais do cruzamento de médias curtas com o preço
        timeSeries = TimeSeries(backtest)
        timeSeries.time_series_short(start_date, end_date, backtest.get_short_ema(), str(setup.short))  
    else:
        strategy = f'{setup.get_strategy()}_min={setup.rsi_min}_max={setup.rsi_max}'
        # atualiza os componentes(widgets) com os ganhos calculados
        trade_gain_rsi_fixed_txt.value = '{:.2f}'.format(backtest.bt_result.get_trade_returns()) 
        accuracy_rsi_txt.value = '{:.2f}'.format(backtest.bt_result.get_accuracy())
        # Chama a função para plotar as séries temporais do RSI
        timeSeries = TimeSeries(backtest)
        timeSeries.rsi_series(start_date, end_date, setup.rsi_min, setup.rsi_max, str(setup.rsi_period), backtest.get_rsi_series())   
    
    result_list = []
    result_list.append([strategy, 
                        backtest.bt_result.get_trade_returns(), 
                        backtest.bt_result.get_buy_and_hold_returns(),
                        backtest.bt_result.get_accuracy(),
                        len(df_trades_buy),
                        len(df_trades_sell),
                        len(df_trades_buy + df_trades_sell)])
    
    return result_list
    #return (backtest.bt_result.get_df_wallet(), result_list)
    
# Análise e representação dos dados em forma de gráfico
def on_update_btn_clicked(b):
    try:
        start_date = start_dtp.value.strftime("%Y-%m-%d")
        end_date = end_dtp.value.strftime("%Y-%m-%d")
        back_test_days = (end_dtp.value - start_dtp.value).days
        start_money = int(wallet_txt.value)
        stop_loss = stop_loss_ma_txt.value #stop_loss_price_txt
        stop_gain = stop_gain_ma_txt.value #stop_gain_price_txt
        buy_increase = buy_inc_txt.value
        sell_decrease = sell_dec_txt.value
        
        # limpa a saída e insere novamente a caixa contendo os controles
        clear_output()
        display(box)
        enable_buttons(False) # desabilita os botões durante o processamento
        print(f'Estratégia Média Curta Exponencial ({short_txt.value}) X Média Longa Simples ({long_txt.value}):')
        config.set_file('../algotradingpy/tests/config_tcc.json')
    
        data = Mysql(db_host=config.get_db_host(), db_port=config.get_db_port(), db_user=config.get_db_user(),
                     db_pass=config.get_db_pass(), db_name=config.get_db_name())
        
        asset = data.get_candles(symbol=coins_dtp.value, time_frame=time_frame_dtp.value, 
                      start_date=start_date, end_date=end_date)
        setup = Setup(strategy='MAxMA', start_money=start_money, short=short_txt.value, long=long_txt.value, 
                      stop_loss=stop_loss, stop_gain=stop_gain, trailing_stop=trailing_stop_ma_chk.value, 
                      buy_increase=buy_increase, sell_decrease=sell_decrease)
        
        result_list_ma = execute_backtest(asset, setup, start_date, end_date, back_test_days)
        
        print('Estratégia Média Curta Exponencial (8) X Média Longa Simples (13):')
        setup = Setup(strategy='MAxMA', start_money=start_money, short=8, long=13, 
                      stop_loss=stop_loss, stop_gain=stop_gain, trailing_stop=trailing_stop_ma_chk.value, 
                      buy_increase=buy_increase, sell_decrease=sell_decrease)
        asset.updated = True
        
        result_list_ma2 = execute_backtest(asset, setup, start_date, end_date, back_test_days)
        
        print(f'Estratégia Média Curta Exponencial ({short_price_txt.value}) X Preço :')     
        setup = Setup(strategy='MAxPrice', start_money=start_money, short=short_price_txt.value, 
                      stop_loss=stop_loss_price_txt.value, stop_gain=stop_gain_price_txt.value, 
                      trailing_stop=trailing_stop_price_chk.value, buy_increase=buy_increase, 
                      sell_decrease=sell_decrease)
        asset.updated = True
        
        result_list_price = execute_backtest(asset, setup, start_date, end_date, back_test_days)
        
        print('Estratégia RSI Min/Max') 
        setup = Setup(strategy='RSI_Min_Max', start_money=start_money, buy_increase=buy_increase, sell_decrease=sell_decrease,
                      rsi_min=float(rsi_min_txt.value), rsi_max=float(rsi_max_txt.value), rsi_period=rsi_period_txt.value)
        asset.updated = True
        result_list_rsi = execute_backtest(asset, setup, start_date, end_date, back_test_days)
        
        #Exportando resultados para arquivo csv
        result_list = result_list_ma + result_list_ma2 + result_list_price + result_list_rsi
        df_export = pd.DataFrame(result_list)
        df_export.columns = ([f'Estratégia ({asset.symbol})', 'Retorno Trade', 'Buy Hold', 'Precisão', 'Trades Compra', 'Trades Venda', 'Trades Total'])
        df_export.set_index(f'Estratégia ({asset.symbol})', inplace=True)  
        export_csv(asset.symbol, start_date, end_date, df_export)
        
        enable_buttons(True)
        
    except:
        enable_buttons(True)
        do_log('Erro ao atualizar o gráfico. Causa:',  sys.exc_info()[0])
        traceback.print_exc()
        
    
def export_csv(symbol, start_date, end_date, df_export_csv):
    # aplica o setup encontrado de forma decrescente, do dia day-daypast até day.
    #df_export_csv = df_export_csv.append(df_simulation)
    #df_export_csv = DataFrame(df_simulation, copy=True)
    
        
    #exporta os dados no formato csv:
    if df_export_csv is not None:
        symbol = str(symbol).replace(':', '')
        file_name= f'{symbol}_{start_date}_a_{end_date}.csv'
        export_csv = df_export_csv.to_csv (r'./' + file_name, index = True, header=True) 
        print()
        print(f'Arquivo {file_name} exportado com sucesso!')
        print('Clique no link para acessar: ')
        display(FileLink('./' + file_name))
        #local_files = FileLinks("./csv")
        #display(local_files)
    else:
        print('\033[1;31mNão foi possível exportar o arquivo csv!\033[m')
    

In [4]:
#### adicionando os controles para interagir com o gráfico
style = {'description_width': 'initial'}
start_dtp = widgets.DatePicker(value=date(2022,10,1), 
                        description='Data inicial: ', disabled=False)
end_dtp = widgets.DatePicker(value=date(2022,10,31), description='Data final: ', disabled=False)
time_frame_dtp = widgets.Dropdown(options=time_frames, value='1h',  description='Intervalo: ', layout=Layout(width='150px', height='30px'))
coins_dtp = widgets.Dropdown(options=coins, value='xrpusd',  description='Moedas: ', layout=Layout(width='200px', height='30px'))
short_txt = widgets.Text(value='13', description='Média Curta: ',  # testar com 8
                         layout=Layout(width='auto', grid_area='lmain1'))
short_price_txt = widgets.Text(value='9',  description='Média Curta: ', 
                         layout=Layout(width='auto', grid_area='rmain1'))
long_txt = widgets.Text(value='21',  description='Média Longa: ', #testar com 13
                         layout=Layout(width='auto', grid_area='lmain2'))
# estratégia de stop, se marcado, o stop loss e gain acompanha a subida do preço
trailing_stop_ma_chk = widgets.Checkbox(value=False, description='Stop Móvel', style=style,
                         layout=Layout(width='auto', grid_area='lmain3'))
trailing_stop_price_chk = widgets.Checkbox(value=False, description='Stop Móvel', style=style,
                         layout=Layout(width='auto', grid_area='rmain2'))
stop_gain_ma_txt = widgets.BoundedFloatText(min = 0.1, max=100, step=0.1, value=3, 
                                         description='Stop Gain: ', style=style,
                         layout=Layout(width='auto', grid_area='lmain4'))
stop_gain_price_txt = widgets.BoundedFloatText(min = 0.1, max=100, step=0.1, value=3, 
                                         description='Stop Gain: ', style=style,
                         layout=Layout(width='auto', grid_area='rmain4'))
stop_loss_ma_txt = widgets.BoundedFloatText(min = -100, max=0, step=0.1, value=-5, 
                                         description='Stop Loss: ', style=style,
                         layout=Layout(width='auto', grid_area='lmain5'))
stop_loss_price_txt = widgets.BoundedFloatText(min = -100, max=0, step=0.1, value=-5, 
                                         description='Stop Loss: ', style=style,
                         layout=Layout(width='auto', grid_area='rmain5'))
trade_gain_mm_txt = widgets.Text(description='Ganhos (%): ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='lmain6'))
trade_gain_mp_txt = widgets.Text(description='Ganhos (%): ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain6'))
buy_hold_gain_txt = widgets.Text(description='Buy & Hold: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain10'))
accuracy_lbl = widgets.Label(description='', value="Precisão (%)", style=style,
                         layout=Layout(width='auto', grid_area='rmain13'))
accuracy_ma_txt = widgets.Text(description='MAxMA: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain14'))
accuracy_price_txt = widgets.Text(description='MAxPrç: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain15'))
accuracy_rsi_txt = widgets.Text(description='RSI: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain16'))
accuracy_rsi_quartiles_txt = widgets.Text(description='C1: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain17'))
accuracy_rsi_outliers1_txt = widgets.Text(description='C2: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain18'))
accuracy_rsi_outliers2_txt = widgets.Text(description='C3: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='rmain19'))
export_lbl = widgets.Label(description='', value="Exportação para arquivo .csv (diretório /csv)", style=style,
                         layout=Layout(width='auto', grid_area='rmain20'))
export_days_txt = widgets.BoundedIntText(min = -30, max=0, step=1, value=-2,
                                     description='Dias p/ trás: ', style=style,
                         layout=Layout(width='auto', grid_area='rmain21'))
export_lbl_var = widgets.Label(description='', value="* É calculada entre o elem. atual e o #º var anterior.", style=style,
                         layout=Layout(width='auto', grid_area='rmain22'))
export_var1_txt = widgets.Text(value='5', description='Variação 1 *: ', placeholder='períodos', style=style,
                         layout=Layout(width='auto', grid_area='rmain23'))
export_var2_txt = widgets.Text(value='10',description='Variação 2 *: ', placeholder='períodos', style=style,
                         layout=Layout(width='auto', grid_area='rmain24'))
export_var3_txt = widgets.Text(value='50',description='Variação 3 *: ', placeholder='períodos', style=style,
                         layout=Layout(width='auto', grid_area='rmain25'))
rsi_period_txt = widgets.Text(value='13',  description='RSI Períodos: ', style=style,
                         layout=Layout(width='auto', grid_area='lmain7'))
rsi_min_txt = widgets.Text(value='20',  description='RSI Min: ', style=style,
                         layout=Layout(width='auto', grid_area='lmain8'))
rsi_max_txt = widgets.Text(value='80', description='RSI Max: ', style=style,
                         layout=Layout(width='auto', grid_area='lmain9'))
trade_gain_rsi_fixed_txt = widgets.Text(description='Ganhos (%): ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='lmain10'))
trade_gain_rsi_quartiles_txt = widgets.Text(description='Ganhos C1: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='lmain11'))
trade_gain_rsi_outliers1_txt = widgets.Text(description='Ganhos C2: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='lmain12'))
trade_gain_rsi_outliers2_txt = widgets.Text(description='Ganhos C3: ', placeholder='%', style=style,
                         layout=Layout(width='auto', grid_area='lmain13'))
legend_rsi1 = widgets.Label(description='', value="* C1 (Cenário 1) = Quartil-Inferior/Quartil-Superior" , style=style,
                         layout=Layout(width='auto', grid_area='lmain14'))
legend_rsi2 = widgets.Label(description='', value="* C2 (Cenário 2) = Outlier-Inferior/Outlier-Superior" , style=style,
                         layout=Layout(width='auto', grid_area='lmain15'))
legend_rsi3 = widgets.Label(description='', value="* C3 (Cenário 3) = (QI+Out-I/QS+Out-S)/2" , style=style,
                         layout=Layout(width='auto', grid_area='lmain16'))

buy_inc_txt = widgets.BoundedFloatText(min = 0, max=1, step=0.01, value=0.0, 
                                         description='Compra (%): ', style=style,
                         layout=Layout(width='auto', grid_area='rmain7'))
sell_dec_txt = widgets.BoundedFloatText(min = -1, max=0, step=0.01, value=0.0, 
                                         description='Venda (%): ', style=style,
                         layout=Layout(width='auto', grid_area='rmain8'))
wallet_txt = widgets.Text(value='500',  description='Wallet (USD): ', placeholder='USD', style=style,
                         layout=Layout(width='auto', grid_area='rmain9'))


blank_wid = widgets.Label()
# declaração dos botões
items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow
update_btn = widgets.Button(description="Executar Backtest", tooltip='Atualiza os gráficos com os valores do \
formulário (não efetua descoberta).')
update_btn.style.button_color = 'lightgreen'
export_btn = widgets.Button(description="Exportar Csv", tooltip='Faz uma exportação dos indicadores regredindo em N dias \
                                                        a partir da data inicial em formato .csv.')
export_btn.style.button_color = 'lightgreen'
# Eventos para os botoões
update_btn.on_click(on_update_btn_clicked)
#export_btn.on_click(on_export_btn_clicked)


# Cria o leiaute para acomodar os componentes Widgets (Primeira parte)
first_col = VBox([start_dtp])
second_col = VBox([end_dtp])
third_col = VBox([time_frame_dtp])
fourth_col = VBox([coins_dtp])
box1 = HBox([first_col, second_col, third_col,  fourth_col])

# Usando gridlayout para melhorar a disposição dos widgets (Segunda parte)

header = widgets.Button(description='Cruzamento - Média x Média',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
footer = widgets.Button(description='',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='lightblue'))
left = GridBox(children=[header, short_txt, long_txt, trailing_stop_ma_chk, stop_gain_ma_txt,
                         stop_loss_ma_txt, trade_gain_mm_txt, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto auto',
            grid_template_columns='28% 28% 30%',
            grid_template_areas='''
            "header header header header"
            "lmain1 lmain2 lmain3  . "
            "lmain4 lmain5 lmain6  . "
            "footer footer footer footer"
            ''')
       )
header = widgets.Button(description='Cruzamento - Média x Preço',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='olive'))
footer = widgets.Button(description='',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))
right = GridBox(children=[header, short_price_txt, trailing_stop_price_chk, stop_gain_price_txt, 
                          stop_loss_price_txt, trade_gain_mp_txt, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto auto',
            grid_template_columns='28% 28% 30%',
            grid_template_areas='''
            "header header header header"
            "rmain1 rmain2 rmain3  . "
            "rmain4 rmain5 rmain6  . "
            "footer footer footer footer"
            ''')
       )
box2 = HBox([left, right])
# RSI

header = widgets.Button(description='RSI',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='olive'))
footer = widgets.Button(description='',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))
left = GridBox(children=[header, rsi_period_txt, rsi_min_txt, rsi_max_txt, trade_gain_rsi_fixed_txt, 
                         trade_gain_rsi_quartiles_txt, trade_gain_rsi_outliers1_txt, trade_gain_rsi_outliers2_txt,
                         legend_rsi1, legend_rsi2, legend_rsi3, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='28% 28% 30%',
            grid_template_areas='''
            "header header header header"
            "lmain7 lmain8 lmain9  . "
            "lmain10 . . . "
            "lmain11 lmain12 lmain13 . "
            "lmain14 lmain14 lmain14 lmain14 "
            "lmain15 lmain15 lmain15 lmain15 "
            "lmain16 lmain16 lmain16 lmain16 "
            " . . . . "
            "footer footer footer footer"
            ''')
       )

# Setups e Buy Hold

header = widgets.Button(description='Setups, Precisão e Exportação',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
footer = widgets.Button(description='',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='lightblue'))
right = GridBox(children=[header, buy_inc_txt, sell_dec_txt, wallet_txt, buy_hold_gain_txt, accuracy_lbl, 
                          accuracy_ma_txt, accuracy_price_txt, accuracy_rsi_txt, accuracy_rsi_quartiles_txt,
                          accuracy_rsi_outliers1_txt, accuracy_rsi_outliers2_txt, export_lbl, export_days_txt, 
                          export_lbl_var, export_var1_txt, export_var2_txt, export_var3_txt, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='28% 28% 30%',
            grid_template_areas='''
            "header header header header"
            "rmain7 rmain8 rmain9  . "
            "rmain10 . . . "
            "rmain13 . . . "
            "rmain14 rmain15 rmain16  . "
            "rmain17 rmain18 rmain19  . "
            "rmain20  rmain20 rmain20 rmain20 "
            "rmain21 rmain22 rmain22 rmain22 "
            " rmain23 rmain24 rmain25 . "
            "footer footer footer footer"
            ''')
       )

box3 = HBox([left, right])

# Mais uma caixa horizontal (Terceira e última parte)
#first_col = VBox([buy_inc_txt, update_btn])
#second_col = VBox([sell_dec_txt, find_btn])
#third_col = VBox([blank_wid, fast_search_chk])
#fourth_col = VBox([wallet_txt, buy_hold_gain_txt])
box4 = HBox([update_btn, export_btn])
# Junta todas as caixas horizontais em uma única caixa vertical
box = VBox([box1, box2, box3, box4])
# definindo as formações para as datas
hour_minute_fmt = mdates.DateFormatter('%H:%M')     
day_hour_fmt = mdates.DateFormatter('%d/%m %H:%M')     
day_month_fmt = mdates.DateFormatter('%d/%m')
year_month_fmt = mdates.DateFormatter('%m/%y')

#print(df.head())
#print(df.dtypes)
# consulta dados de setup salvos no banco


In [5]:
verbose_mode = False
display(box)

#InteractiveShell.ast_node_interactivity = "all"        

# Remova os comentários das linhas abaixo para instalar as dependências desse notebook
#!pip3 install --user ipywidgets
#!pip3 install --user IPython